---

# PROJECT: BOOK REVIEW AND INFORMATION DATABASE

### AIM: Create Pipeline Script to collect interested data from books database
<img src="C:/Users/lamxu/Desktop/git demo/database_graph.png" alt=" ">
[Data Diagram](https://drive.google.com/file/d/1uHggMOUo4yb4Dn4eQVQ287vE23TYCyHi/view?usp=sharing)
    
### TABLE OF CONTENT:

* [Overview Tables](#part_0)  

* [Task_1> Number of books released after specific date](#part_1) 

* [Task_2> Book number of reviews and average rating](#part_2) 

* [Task_3> Publishers with greatest number of books](#part_3) 

* [Task_4> Authors with the highest average book rating](#part_4) 

* [Task_5> User average number of text reviews](#part_5) 

---
---

In [1]:
# FUNCTIONS USED FOR PROJECT-----------------------------------------------------------#

def Books_Database(query):
    # import libraries
    import pandas as pd
    from sqlalchemy import create_engine

    # Database connection parameters
    db_config = {'user': 'praktikum_student',          # user name
                'pwd': 'Sdf***********',               # password
                'host': '******.mdb.yandexcloud.net',
                'port': 6***,                          # connection port
                'db': 'data-analyst-final-project-db'} # the name of the data base

    # Connection string
    connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(   db_config['user'],
                                                                db_config['pwd'],
                                                                db_config['host'],
                                                                db_config['port'],
                                                                db_config['db']     )
    # Connection engine
    engine = create_engine(connection_string, connect_args={'sslmode':'require'})

    # Connect and get data!
    return pd.io.sql.read_sql(query, con = engine)


def Breakline(n=100):
    """
    Draw breakline
    """
    print()
    print('-'*n)
    print()

## Overview Tables <a id="part_0"></a>

In [2]:
tables = ['books', 'authors', 'ratings', 'reviews', 'publishers']

dfs = {}
for table in tables:
    query = ''' SELECT * FROM {}'''.format(table)
    dfs[table] = Books_Database(query)

    print('_'*10, '"{}" TABLE'.format(table.upper()), '_'*10)
    print()

    print(dfs[table].info())
    display(dfs[table].head(5))
    Breakline()

__________ "BOOKS" TABLE __________

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
None


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



----------------------------------------------------------------------------------------------------

__________ "AUTHORS" TABLE __________

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
None


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



----------------------------------------------------------------------------------------------------

__________ "RATINGS" TABLE __________

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
None


...


----------------------------------------------------------------------------------------------------

__________ "REVIEWS" TABLE __________

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
None


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...



----------------------------------------------------------------------------------------------------

__________ "PUBLISHERS" TABLE __________

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
None


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



----------------------------------------------------------------------------------------------------



## Task_1: Number of books released after specific date <a id="part_1"></a>

#### Result: 819 books was released after 2000-01-01

In [3]:
limit_date = '2000-01-01'

query1 = '''
        SELECT  COUNT(DISTINCT book_id)
        FROM    books
        WHERE   publication_date > '{}'
        ;
        '''.format(limit_date)

print('NUMBER OF BOOK RELEASE AFTER', limit_date)
print(Books_Database(query1))

NUMBER OF BOOK RELEASE AFTER 2000-01-01
   count
0    819


## Task_2: Book number of reviews and average rating <a id="part_2"></a>

#### RESULT: New table show number_of_reviews and average_rating for each book_id/title 

In [4]:
query2 = '''
        SELECT  books.book_id, title, number_of_reviews, average_rating
        FROM    books

                LEFT JOIN ( SELECT      book_id, COUNT(review_id) AS number_of_reviews
                            FROM        reviews
                            GROUP BY    book_id ) AS reviews2 ON reviews2.book_id = books.book_id

                LEFT JOIN ( SELECT      book_id, AVG(rating) AS average_rating
                            FROM        ratings
                            GROUP BY    book_id ) AS ratings2 ON ratings2.book_id = books.book_id
        ORDER BY book_id
        ;
        '''

print('NUMBER OF REVIEWS AND AVERAGE RATING OF EACH BOOK')
display(Books_Database(query2))

NUMBER OF REVIEWS AND AVERAGE RATING OF EACH BOOK


,book_id,title,number_of_reviews,average_rating
0,1,'Salem's Lot,2.0,3.666667
1,2,1 000 Places to See Before You Die,1.0,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3.0,4.666667
3,4,1491: New Revelations of the Americas Before C...,2.0,4.500000
4,5,1776,4.0,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3.0,3.666667
996,997,Xenocide (Ender's Saga #3),3.0,3.400000
997,998,Year of Wonders,4.0,3.200000
998,999,You Suck (A Love Story #2),2.0,4.500000


## Task_3: Publishers with greatest number of books <a id="part_3"></a>

#### RESULT: "Penguin Books" publisher has the greatest number of more than 50 pages books.

In [5]:
min_pages = 50
top_n = 5

query3 = '''
        SELECT  books.publisher_id, publisher, COUNT(DISTINCT book_id) AS number_of_books
        FROM    books
                LEFT JOIN publishers ON books.publisher_id = publishers.publisher_id
        WHERE   num_pages > {}
        GROUP BY books.publisher_id, publisher
        ORDER BY number_of_books DESC, publisher_id
        LIMIT   {}
        ;
        '''.format(min_pages, top_n)

print('TOP_{} PUBLISHERS WITH HIGHEST NUMBER OF BOOKS (more than {} pages books only)'.format(top_n, min_pages))
Books_Database(query3)

TOP_5 PUBLISHERS WITH HIGHEST NUMBER OF BOOKS (more than 50 pages books only)


,publisher_id,publisher,number_of_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,33,Ballantine Books,19


In [6]:
# ALTERNATIVE QUERY TO IDENTIFY TOP PUBLISHER
# if more than 1 publisher have the same maximal number of books, all will be showed

min_pages = 50 

query3b = '''
        SELECT  books.publisher_id, publisher, COUNT(DISTINCT book_id) AS number_of_books
        FROM    books
                LEFT JOIN publishers ON books.publisher_id = publishers.publisher_id
        WHERE   num_pages > {}
        GROUP BY books.publisher_id, publisher
        HAVING  COUNT(DISTINCT book_id) =
                                (   SELECT MAX(count.nob)
                                    FROM (  SELECT  publisher_id, COUNT(DISTINCT book_id) AS nob
                                            FROM    books
                                            WHERE   num_pages > {}
                                            GROUP BY publisher_id   ) AS count )
        ORDER BY publisher_id                
        ;
        '''.format(min_pages, min_pages)

print('PUBLISHER WITH HIGHEST NUMBER OF BOOKS (more than {} pages books only)'.format(min_pages))
Books_Database(query3b)

PUBLISHER WITH HIGHEST NUMBER OF BOOKS (more than 50 pages books only)


,publisher_id,publisher,number_of_books
0,212,Penguin Books,42


## Task_4: Authors with the highest average book rating <a id="part_4"></a>

#### RESULT: "J.K. Rowling/Mary GrandPré" has the highest average rating of books having at least 50 rating.

In [7]:
min_number_rating = 50
top_n = 5

query4 = '''
        SELECT  books.author_id, author, AVG(book_rating) AS average_book_rating
        FROM    books
                INNER JOIN (   SELECT   book_id,
                                        COUNT(rating) AS number_of_rating,
                                        AVG(rating) AS book_rating
                                FROM    ratings
                                GROUP BY book_id
                                HAVING  COUNT(rating) >= {}  ) AS book_rating 
                            ON books.book_id = book_rating.book_id
                
                LEFT JOIN authors ON books.author_id = authors.author_id
        GROUP BY books.author_id, author
        ORDER BY average_book_rating DESC
        LIMIT   {}
        ;
        '''.format(min_number_rating, top_n)

print('TOP {} AUTHORS WITH HIGHEST AVERAGE BOOK RATINGS (at least {} rating books only)'.format(top_n, min_number_rating))
Books_Database(query4)

TOP 5 AUTHORS WITH HIGHEST AVERAGE BOOK RATINGS (at least 50 rating books only)


,author_id,author,average_book_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844
1,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,240,J.R.R. Tolkien,4.258446
3,376,Louisa May Alcott,4.192308
4,498,Rick Riordan,4.080645


## Task_5: User average number of text reviews <a id="part_5"></a>

#### RESULT: Users who rated more than 50 books make around 24 reviews on average.

In [8]:
min_book_rated = 50

query5 = '''
        SELECT  AVG(number_of_review)
        FROM    (   SELECT  username, COUNT(DISTINCT book_id) AS number_book_rated
                    FROM    ratings
                    GROUP BY username
                    HAVING  COUNT(DISTINCT book_id) > {}    ) AS user_rating

                LEFT JOIN

                (   SELECT  username, COUNT(review_id) AS number_of_review
                    FROM    reviews
                    GROUP BY username   ) AS user_review
                
                ON user_rating.username = user_review.username
        ;
        '''.format(min_book_rated)

print('USER AVERAGE NUMBER OF TEXT REVIEWS (users rated more than {} books only)'.format(min_book_rated))
Books_Database(query5)

USER AVERAGE NUMBER OF TEXT REVIEWS (users rated more than 50 books only)


,avg
0,24.333333
